# UMLS data wrangling from data and knowledge source


### Purpose: 
The purpose of this notebook is to do data wrangling from patient datasource and UMLS knowledge source using numbpy or pandas

### Input: 
Datasource: patient data as a XML file
Knowledge source:  UMLS datafiles in RRF format

### Output:
A csv file which has extracted information from input files including MRN,Medication CUI, Medication Name, Drug Class, Diagnosis, and Mechanism of Action for the drug.

### Steps:
1. Fist import all the relavant libaries,read input files,check few rows of input files.

2. Read  XML file and extract each MRN and corrosponding Medication CUI from each record and store this information into empty dictionary. Append this dictionary to empty list and convert list into dataframe.

3. Convert MMREL file to dataframe and merge this dataframe to dataframe created in step 2 on Medication CUI and generate merged dataframe.

4. Convert MRCONSO file dataframe and merge this dataframe to dataframe created in step 2 on Medication CUI and generate merged dataframe..

5. Filter the dataframe created on step 3 to create dataframe for drug class, mechanism of action and diagnosis.

6. Extract information for drug class, diagnosis, and mechanism of action using corropsponding drug concept identifier from MRCONSO file and store it in dataframes

7. Merge the generated dataframe from step3 and step 6 and export output to csv file format.

## 1. Loading list of documents and read XML,MRREL, MRCONSO files

In [ ]:
#Import relevant dependancies
import os
import csv
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

#File path
dataset_path="/med/dataset.xml"
mrconso_path="/med/MRCONSO.RRF"
mrrel_path= "/med/MRREL.RRF"

# Read MRREL file using pandas script and check the data
dataframe_mmrel=pd.read_csv(mrrel_path,header=None,sep="|")

# Read MRCONSO file using pandas script and check the data
dataframe_mrconso=pd.read_csv(mrconso_path,header=None,sep="|")

#Read XML file check the data
tree=ET.parse("/med/dataset.xml")
root=tree.getroot()

## 2. Parse patient MRN and Medication CUI from XML file

We are going to parse the XML file and extract MRN and Medication information and convert it inot dataframe.

In [ ]:
#Create global variable.
med_list=[]                                     #Create empty  list to hold MRN and medication name

#Iteration thorugh all the nodes of tree using nested for loop
for child in root:
    for subchild in child:
        for stepchild in subchild:
            med_dict=dict()                      #Create empty dictinary to store MRN and medication as key-value pairs
            if stepchild.tag=="MRN":
                mrn_value=stepchild.attrib.get("value")
                med_dict["MRN"]=mrn_value
                #d.append(c)
            if stepchild.tag=="Item":
                med_value=stepchild.attrib.get("value")
                med_dict["Medication_CUI"]=med_value
                med_dict["MRN"]=mrn_value
                med_list.append(med_dict)  
#Create a dataframe from list
mrnmed=pd.DataFrame(med_list)

##  3. Parse relavant data from MRCONSO.RRF file 

We are going to parse the MRCONSO file convert to dataframe. This newly generated dataframe will be use later to extract drug class, diagnosis, and mechanism of action information from it.

In [ ]:
#Slice the relavant column from MRCONSOfile
mrconso_df=dataframe_mrconso.loc[:,[0,14]]

#Rename the column name
mrconso_df=mrconso_df.rename(index=str,columns={0:"Medication_CUI",14:"Medication_Name"})

#Merge mrn_med and mrconso_df on Medication_CUI
mrnmed_mrconso=pd.merge(mrnmed,mrconso_df, on=["Medication_CUI"],how="left",left_index=True)
mrnmed_mrconso.reset_index(drop=True,inplace=True)

##  4. Parse relavant data from MRREL.RRF file 

We are going to parse the MRREL file and extract CUI1, CUI2, Relationship(REL), and Additonal relation (RELA) for drug. This newly generated dataframe will be use later to parse drug class, diagnosis, and mechanism of action from it.

In [ ]:
#Slice the relavant column from MRREL file
mmrel_df=dataframe_mmrel.loc[:,[0,3,4,7]]

#Rename the column name
mmrel_df=mmrel_df.rename(index=str,columns={0:"Medication_CUI", 3:"REL", 4:"CUI2", 7:"RELA"})
#Merge dataframe
mrnmed_mmrel=pd.merge(mrnmed,mmrel_df, on=["Medication_CUI"],how="inner")

## 5. Create dataframe for Drug class, Diagnosis, Mechanism of Action

We are going create dataframe for drug class, diagnosis, and mechanism of action using relavant term from MRREL file 

In [ ]:
#Create dataframe for Diagnosis
diagnosis_df=mrnmed_mmrel.loc[(mrnmed_mmrel["RELA"]=="may_be_treated_by")]
diagnosis_df.reset_index(drop=True,inplace=True)

#Create dataframe for Mechanism of action
mechanism_df=mrnmed_mmrel.loc[(mrnmed_mmrel["RELA"]=="mechanism_of_action_of")]
mechanism_df.set_index(["MRN"],inplace=True,drop=True)
mechanism_df.reset_index(inplace=True,drop=False)

#Create dataframe for drug class by using MRREL relationship term
drug_df=mrnmed_mmrel.loc[(mrnmed_mmrel["REL"]=="CHD")|(mrnmed_mmrel["REL"]=="member_of")|(mrnmed_mmrel["REL"]=="PAR")|(mrnmed_mmrel["RELA"]=="isa")
                     |(mrnmed_mmrel["RELA"]=="inverse_isa")]

## 6. Extract information from MRCONSO for drug class, diagnosis, and mechanism of action


In [ ]:
#Extract the information from MRCONSO file using the CUI value corroponding ConceptID for diagnosis
diagnosis_mrconso=pd.merge(diagnosis_df, mrconso_df,left_on="CUI2",right_on="Medication_CUI",left_index=True)
diagnosis_mrconso.reset_index(drop=True,inplace=True)
diagnosis_mrconso=diagnosis_mrconso.drop(["REL","CUI2","RELA","Medication_CUI_y"],axis=1)
diagnosis_mrconso=diagnosis_mrconso.rename(index=str, columns={"Medication_CUI_x":"Medication_CUI",
                                                                "Medication_Name":"Medication_Diagnosis"})

#Extract the information from MRCONSO file using the CUI1 value corroponding ConceptID for mechanism of action
mechanism_mrconso=pd.merge(mechanism_df,mrconso_df,left_on="CUI2",right_on="Medication_CUI",left_index=True,how="inner")
mechanism_mrconso.reset_index(drop=True,inplace=True)
mechanism_mrconso=mechanism_mrconso.drop(["REL","CUI2","RELA","Medication_CUI_y"],axis=1)

mechanism_mrconso=mechanism_mrconso.rename(index=str, columns={"Medication_CUI_x":"Medication_CUI",
                                                              "Medication_Name":"Medication_Mechanism_of_Action"})

#Extract the information from MRCONSO file using the CUI1 value corroponding ConceptID for drugclass
drug_mrconso=pd.merge(drug_df,mrconso_df,left_on="CUI2",right_on="Medication_CUI",left_index=True)
drug_mrconso.reset_index(drop=True,inplace=True)
drug_mrconso=drug_mrconso.drop(["REL","CUI2","RELA","Medication_CUI_y"],axis=1)
drug_mrconso=drug_mrconso.rename(index=str, columns={"Medication_CUI_x":"Medication_CUI",
                                                              "Medication_Name":"Medication_class"})

## 7. Merge the extracted dataframes from MRREL and MRCONSO file for drugclass, diagnosis and mechanism of action and export output to csv file

In [ ]:
#Merge first MRN-Medication CUI and Drug class dataframe
df_1=pd.merge(mrnmed_mrconso,drug_mrconso)

#Merge Mechanism of action and Diagnsosis dataframe
df_2=pd.merge(mechanism_mrconso,diagnosis_mrconso)

#Merge df_1 and df_1
final_df=pd.merge(df_1,df_2,how="inner")

#Drop duplicates
final_output=final_df.drop_duplicates(subset=['MRN',"Medication_CUI","Medication_Name","Medication_class"])
final_output.reset_index(drop=True,inplace=True)

#Print final output to check data
print(final_output.head(2))

#export dataframe to csv formate
final_csv=final_output.to_csv("output_cp2.csv", encoding='utf-8',index=False)